In [32]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [33]:
normalized_data = pd.read_csv("normalized_stock_data.csv")

print(f"there are {len(normalized_data)} rows")
normalized_data.head()

there are 262 rows


,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,0.002198,0.000487,0.006397,0.001871,0.000000,0.4,0.0
1,0.001843,0.000000,0.003596,0.001100,0.269540,0.0,0.0
2,0.000939,0.002022,0.005530,0.003165,0.277210,0.0,0.0
3,0.003480,0.003948,0.008077,0.005852,0.386136,0.0,0.0
4,0.006016,0.004188,0.010104,0.005416,0.132201,0.0,0.0


In [34]:
# create segmented sequences
def create_sequence(data, seq_len):
    sequences = []
    for i in range(len(data) - seq_len):
        sequences.append(data[i : i + seq_len])
    return np.array(sequences)

sequence_length = 3
sequences = create_sequence(normalized_data, sequence_length)
# [[row1, row2, row3], .... []]

print(len(sequences))

259


In [35]:
# train-test split (70-30 split)
split_ratio = 0.7
train_size = int(len(sequences) * split_ratio)

print(f"train size: {train_size}")
print(f"test size: {len(sequences) - train_size}")

train_sequences = sequences[:train_size]
test_sequences = sequences[train_size:]

print(f"train sequences: {len(train_sequences)}")
print(f"test sequences: {len(test_sequences)}")

train size: 181
test size: 78
train sequences: 181
test sequences: 78


In [36]:
# Key parameters
seq_length = 60  # Sequence length
feature_dim = 7  # Number of features
hidden_dim = 128  # Hidden units in LSTM and RNN
batch_size = 32

In [42]:
# Generator: LSTM
class Generator(nn.Module):
    def __init__(self, feature_dim, hidden_dim, seq_length):
        super(Generator, self).__init__()
        self.lstm = nn.LSTM(feature_dim, hidden_dim, batch_first=True) 
        self.fc = nn.Linear(hidden_dim, feature_dim) # fully connected layer
        self.tanh = nn.Tanh() # tanh activation function
        self.seq_length = seq_length
    
    def forward(self, x):
        hidden_state = torch.zeros(1, x.size(0), hidden_dim).to(x.device)
        cell_state = torch.zeros(1, x.size(0), hidden_dim).to(x.device)
        output, _ = self.lstm(x, (hidden_state, cell_state))
        output = self.fc(output[:, -1, :])  # Fully connected layer on the last time step
        return self.tanh(output)


In [38]:
# Discriminator: RNN
# TODO: 

In [44]:
# Instantiate models
device = torch.device("cuba" if torch.cuda.is_available() else "cpu")
generator = Generator(feature_dim, hidden_dim, seq_length).to(device)
# TODO: Discriminator

In [ ]:
criterion = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=0.001)
# TODO: optimizer for discriminator

In [ ]:
def train_GAN(generator, discriminator, data, epochs=1000, batch_size=32):
    real_label = 1
    fake_label = 0

    # repeat training for epochs
    for epoch in range(epochs):
        # mini-batch training
        for _ in range(len(data) // batch_size):
            # Train Discriminator

            # Train Generator
            pass